In [4]:
# Pick GPU.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

from nflows.flows.base import Flow
from nflows.distributions.normal import StandardNormal
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation

from scipy import stats
import numpy as np
import sklearn.metrics as metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split     

np.random.seed(666)

# Load in data
mc = np.load("data/zenodo/Pythia21_Zjet_pTZ-200GeV_0.npz")

mc_pt =  mc['sim_jets'][:, 0] 
mc_eta = mc['sim_jets'][:, 1]
mc_phi = mc['sim_jets'][:, 2]
mc_m =   mc['sim_jets'][:, 3]
mc_w = mc['sim_widths']
mc_sdms = mc['sim_sdms']
mc_mults = mc['sim_mults']
mc_tau21 = mc['sim_tau2s'] / (mc_w + 10**-50)
mc_jm = np.log(mc_sdms**2 / mc_pt**2 + 10**-100)

jm_mask = mc_jm > -20
mc_pt = mc_pt[jm_mask]
mc_eta = mc_eta[jm_mask]
mc_phi = mc_phi[jm_mask]
mc_m = mc_m[jm_mask]
mc_w = mc_w[jm_mask]
mc_sdms = mc_sdms[jm_mask]
mc_mults = mc_mults[jm_mask]
mc_tau21 = mc_tau21[jm_mask]
mc_jm = mc_jm[jm_mask]

mc = np.vstack([mc_m, mc_mults, mc_w, mc_jm, mc_tau21, mc_pt]).T
n, d = mc.shape

def calculate_auc(fake, real):
    np.random.seed(666)
    n, d = fake.shape
    y = stats.bernoulli.rvs(0.5, size = n).astype('float32')
    X = np.zeros_like(fake)
    X[y == 0] = real[y == 0]
    X[y == 1] = fake[y == 1]

    # Take 70% of data to be training data.
    X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, train_size = 0.7)
    ab_clf = AdaBoostClassifier()
    ab_clf.fit(X_trn, y_trn)
    
    y_hat = ab_clf.predict_proba(X_tst)[:, 1]
    auc = metrics.roc_auc_score(y_tst, y_hat)
    return auc


# Initialize flow
num_layers = 5
base_dist = StandardNormal(shape=[d])

transforms = []
for _ in range(num_layers):
    transforms.append(ReversePermutation(features=d))
    transforms.append(MaskedAffineAutoregressiveTransform(features=d, 
                                                          hidden_features=8))
transform = CompositeTransform(transforms)
flow = Flow(transform, base_dist)
flow.to(device)


# Calculate AUCs
aucs = np.zeros(100)
for i in range(0):
    ckpt = torch.load('flows/nflows/sim/8/best/ckpt_{}'.format(i))
    flow.load_state_dict(ckpt['model_state_dict'])
    flow.eval()
    
    smp = flow.sample(n).cpu().detach().numpy()
    smp[:, 1] = np.round(smp[:, 1])
    
    aucs[i] = calculate_auc(smp, mc)
    print(aucs[i])
# np.save('sim_8_aucs.npy', aucs)

True
1
0
Tesla V100S-PCIE-32GB
cuda:0
